In [ ]:
### THIS TOP PART IS STARTER CODE FROM ELSE WHERE AND NEEDS TO BE ADJUSTED LATER

In [ ]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

In [ ]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Cloud MySQL Database Connection on AWS
cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")

In [ ]:
# Create a local database engine connection
local_conn = local_engine.connect()

# Create a remote database engine connection
cloud_conn = cloud_engine.connect()

In [ ]:
# Create local gwsis database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)
    
# Connect to local gwsis database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)    

In [ ]:
##Read CSVs to pandas dataframes - DONE
##Renaming of Columns Needed for Better Database Storage - DONE
##Check Data types -- anything need reformatting into integers -DONE
##check for total number of cases - DONE
##create a dataframe with just id and country from both tables
##CONSIDER PARSING COUNTRY TEXT AND DOING A BOOLEAN TEST TO SEE IF SAME AND THEN UPDATE THE REST? (total cases should be 149)
##Remove unwanted cases
##Once unwanted cases are removed, there will be some countries to rename
##Merge Tables
##Push to Cloud as individual and joined table

In [16]:
# Import Pandas
import pandas as pd

In [17]:
happiness_df = pd.read_excel('Happiness.xlsx')
happiness_df.head()

,Index,Country name,Regional indicator,Ladder score,Happiness Rank,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,...,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,1,Afghanistan,South Asia,2.5669,153,0.031311,2.628270,2.505530,7.462861,0.470367,...,-0.096429,0.933687,1.972317,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226,1.507236
1,2,Albania,Central and Eastern Europe,4.8827,105,0.056116,4.992687,4.772713,9.417931,0.671070,...,-0.042309,0.896304,1.972317,0.906653,0.830484,0.846330,0.461946,0.171028,0.025361,1.640897
2,3,Algeria,Middle East and North Africa,5.0051,100,0.044236,5.091802,4.918397,9.537965,0.803385,...,-0.121105,0.735485,1.972317,0.943856,1.143004,0.745419,0.083944,0.118915,0.129191,1.840812
3,4,Argentina,Latin America and Caribbean,5.9747,55,0.053442,6.079446,5.869954,9.810955,0.900568,...,-0.194914,0.842010,1.972317,1.028466,1.372544,0.849774,0.520840,0.070100,0.060415,2.072541
4,5,Armenia,Commonwealth of Independent States,4.6768,116,0.058595,4.791646,4.561953,9.100476,0.757479,...,-0.138780,0.773545,1.972317,0.808262,1.034577,0.775857,0.378076,0.107226,0.104618,1.468162


In [18]:
happy_columns = ['Standard error of ladder score', 'upperwhisker', 'lowerwhisker', 'Dystopia + residual','Ladder score in Dystopia']
happiness_df.drop(happy_columns, inplace=True, axis=1)
happiness_df

,Index,Country name,Regional indicator,Ladder score,Happiness Rank,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
0,1,Afghanistan,South Asia,2.5669,153,7.462861,0.470367,52.590000,0.396573,-0.096429,0.933687,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226
1,2,Albania,Central and Eastern Europe,4.8827,105,9.417931,0.671070,68.708138,0.781994,-0.042309,0.896304,0.906653,0.830484,0.846330,0.461946,0.171028,0.025361
2,3,Algeria,Middle East and North Africa,5.0051,100,9.537965,0.803385,65.905174,0.466611,-0.121105,0.735485,0.943856,1.143004,0.745419,0.083944,0.118915,0.129191
3,4,Argentina,Latin America and Caribbean,5.9747,55,9.810955,0.900568,68.803802,0.831132,-0.194914,0.842010,1.028466,1.372544,0.849774,0.520840,0.070100,0.060415
4,5,Armenia,Commonwealth of Independent States,4.6768,116,9.100476,0.757479,66.750656,0.712018,-0.138780,0.773545,0.808262,1.034577,0.775857,0.378076,0.107226,0.104618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,149,Venezuela,Latin America and Caribbean,5.0532,99,8.977794,0.890408,66.505341,0.623278,-0.169091,0.837038,0.770239,1.348547,0.767026,0.271717,0.087179,0.063625
149,150,Vietnam,Southeast Asia,5.3535,83,8.809546,0.849987,67.952736,0.939593,-0.094533,0.796421,0.718092,1.253075,0.819134,0.650836,0.136489,0.089848
150,151,Yemen,Middle East and North Africa,3.5274,146,7.759683,0.817981,56.727283,0.599920,-0.157735,0.800288,0.392702,1.177477,0.415000,0.243721,0.094689,0.087352
151,152,Zambia,Sub-Saharan Africa,3.7594,141,8.224720,0.698824,55.299377,0.806500,0.078037,0.801290,0.536834,0.896037,0.363593,0.491318,0.250621,0.086705


In [20]:
happiness_df.rename(columns={'Country name': 'Country_Happy', 'Index': 'Happy_Index', 'Regional indicator': 'Region','Ladder score': 'Ladder_score', 'Happiness Rank': 'Happiness_rank', 'Logged GDP per capita': 'GDP', 'Social support': 'social_support', 'Healthy life expectancy':'Life_expectancy',
                            'Perceptions of corruption': 'Corruption', 'Freedom to make life choices': 'Freedom','Explained by: Log GDP per capita': 'Exp_GDP', 'Explained by: Social support': 'Exp_support','Explained by: Healthy life expectancy': 'Exp_health', 'Explained by: Freedom to make life choices':'Exp_freedom', 
                            'Explained by: Generosity': 'Exp_generosity', 'Explained by: Perceptions of corruption': 'Exp_corruption'}, inplace=True)
happiness_df.head()

,Happy_Index,Country_Happy,Region,Ladder_score,Happiness_rank,GDP,social_support,Life_expectancy,Freedom,Generosity,Corruption,Exp_GDP,Exp_support,Exp_health,Exp_freedom,Exp_generosity,Exp_corruption
0,1,Afghanistan,South Asia,2.5669,153,7.462861,0.470367,52.590000,0.396573,-0.096429,0.933687,0.300706,0.356434,0.266052,0.000000,0.135235,0.001226
1,2,Albania,Central and Eastern Europe,4.8827,105,9.417931,0.671070,68.708138,0.781994,-0.042309,0.896304,0.906653,0.830484,0.846330,0.461946,0.171028,0.025361
2,3,Algeria,Middle East and North Africa,5.0051,100,9.537965,0.803385,65.905174,0.466611,-0.121105,0.735485,0.943856,1.143004,0.745419,0.083944,0.118915,0.129191
3,4,Argentina,Latin America and Caribbean,5.9747,55,9.810955,0.900568,68.803802,0.831132,-0.194914,0.842010,1.028466,1.372544,0.849774,0.520840,0.070100,0.060415
4,5,Armenia,Commonwealth of Independent States,4.6768,116,9.100476,0.757479,66.750656,0.712018,-0.138780,0.773545,0.808262,1.034577,0.775857,0.378076,0.107226,0.104618


In [27]:
happiness_df.dtypes

Happy_Index          int64
Country_Happy       object
Region              object
Ladder_score       float64
Happiness_rank       int64
GDP                float64
social_support     float64
Life_expectancy    float64
Freedom            float64
Generosity         float64
Corruption         float64
Exp_GDP            float64
Exp_support        float64
Exp_health         float64
Exp_freedom        float64
Exp_generosity     float64
Exp_corruption     float64
dtype: object

In [35]:
len(happiness_df)

153

In [21]:
bank_df = pd.read_excel('UpdatedWBdata.xlsx')
bank_df

,Index,Country,2000 Pop Mil,2019 Pop Mil,Avg Annual Pop Growth 2000-2019,Percent Pop Age 0-14 2019,Percent Pop Age 15-64 2019,Percent Pop Age 65 and up 2019,Dep Ratio Percent Young Working Pop 2019,Dep Ratio Percent Old Working Pop 2019,Crude Death Rate per 1k 2018,Crude birth rate per 1k 2018
0,1,Afghanistan,20.8,38.0,3.2,42.0,55.0,3.0,77.0,5.0,6.0,32.0
1,2,Albania,3.1,2.9,-0.4,17.0,68.0,14.0,25.0,21.0,8.0,12.0
2,3,Algeria,31.0,43.1,1.7,31.0,63.0,7.0,49.0,10.0,5.0,24.0
3,4,American Samoa,0.1,0.1,-0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Andorra,0.1,0.1,0.9,NaN,NaN,NaN,NaN,NaN,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
209,210,Virgin Islands (U.S.),0.1,0.1,-0.1,19.0,61.0,20.0,32.0,33.0,8.0,13.0
210,211,West Bank and Gaza,2.9,4.7,2.5,39.0,58.0,3.0,66.0,5.0,3.0,29.0
211,212,"Yemen, Rep.",17.4,29.2,2.7,39.0,58.0,3.0,68.0,5.0,6.0,30.0
212,213,Zambia,10.4,17.9,2.8,44.0,53.0,2.0,83.0,4.0,6.0,36.0


In [22]:
bank_columns = ['2000 Pop Mil', 'Crude Death Rate per 1k 2018', 'Crude birth rate per 1k 2018', 'Dep Ratio Percent Young Working Pop 2019', 'Dep Ratio Percent Old Working Pop 2019' ]
bank_df.drop(bank_columns, inplace=True, axis=1)
bank_df.head()

,Index,Country,2019 Pop Mil,Avg Annual Pop Growth 2000-2019,Percent Pop Age 0-14 2019,Percent Pop Age 15-64 2019,Percent Pop Age 65 and up 2019
0,1,Afghanistan,38.0,3.2,42.0,55.0,3.0
1,2,Albania,2.9,-0.4,17.0,68.0,14.0
2,3,Algeria,43.1,1.7,31.0,63.0,7.0
3,4,American Samoa,0.1,-0.2,NaN,NaN,NaN
4,5,Andorra,0.1,0.9,NaN,NaN,NaN


In [25]:
bank_df.rename(columns={'Country': 'Country_Bank', 'Index': 'Bank_Index', '2019 Pop Mil': '2019_pop', 'Avg Annual Pop Growth 2000-2019': '10yr_popgrowth_percent', 'Percent Pop Age 0-14 2019': 'pop_per_14under','Percent Pop Age 15-64 2019': 'pop_per_15to64', 'Percent Pop Age 65 and up 2019': 'pop_per_65plus'}, inplace=True)
bank_df.head()

,Bank_Index,Country_Bank,2019_pop,10yr_popgrowth_percent,pop_per_14under,pop_per_15to64,pop_per_65plus
0,1,Afghanistan,38.0,3.2,42.0,55.0,3.0
1,2,Albania,2.9,-0.4,17.0,68.0,14.0
2,3,Algeria,43.1,1.7,31.0,63.0,7.0
3,4,American Samoa,0.1,-0.2,NaN,NaN,NaN
4,5,Andorra,0.1,0.9,NaN,NaN,NaN


In [26]:
bank_df.dtypes

Bank_Index                  int64
Country_Bank               object
2019_pop                  float64
10yr_popgrowth_percent    float64
pop_per_14under           float64
pop_per_15to64            float64
pop_per_65plus            float64
dtype: object

In [46]:
len(bank_df)

214

In [ ]:
### THIS IS STARTER CODE FROM ELSEWHERE AND NEEDS TO BE ADJUSTED LATER

In [ ]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)